In [1]:
# preamble to be able to run notebooks in Jupyter and Colab
try:
    from google.colab import drive
    import sys
    
    drive.mount('/content/drive')
    notes_home = "/content/drive/Shared drives/CSC310/notes/"
    user_home = "/content/drive/My Drive/"
    
    sys.path.insert(1,notes_home) # let the notebook access the notes folder

except ModuleNotFoundError:
    notes_home = "" # running native Jupyter environment -- notes home is the same as the notebook
    user_home = ""  # under Jupyter we assume the user directory is the same as the notebook

# k-NN Classification

k-NN: **k** **N**earest **N**eighbors

Neighbors-based classification is a type of *instance-based learning*: it does not attempt to construct a general internal model, but simply stores instances of the training data. 

Classification is computed from a simple majority vote of the *nearest neighbors of each point*: a query point is assigned the data class which has the most representatives within the nearest neighbors of the point.


## Example

Consider,

<!-- ![knn](assets/knn.png) -->

<img src="https://raw.githubusercontent.com/lutzhamel/ds/master/notes/assets/knn.png" height="256" width="256">

* The test sample (green circle) should be classified either to the class of blue squares or to the class of red triangles. 

* If k = 3 (solid line circle) it is assigned to the class of red triangles because there are 2 triangles and only 1 square inside the inner circle. 

* If k = 5 (dashed line circle) it is assigned to the class of blue squares (3 squares vs. 2 triangles inside the dashed circle).


## k-NN Classification

The training examples are vectors in a multidimensional feature space, each with a class label. 

The training phase of the algorithm consists only of storing the feature vectors and class labels of the training samples.

In the classification phase, k is a user-defined constant, and an unlabeled vector (a query or test point) is classified by assigning the label which is most frequent among the k training samples nearest to that query point.

A commonly used distance metric for continuous variables is the Euclidean distance. For discrete variables, such as for text classification, another metric can be used, such as the Hamming distance.


## A Code Example

In [2]:
import pandas as pd
import numpy as np
np.set_printoptions(formatter={'float_kind':"{:3.2f}".format})
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# get data
df = pd.read_csv(notes_home+"assets/iris.csv")
X  = df.drop(['id','Species'],axis=1)
y = df['Species']

# set up the model with k=3
model = KNeighborsClassifier(n_neighbors=3)

# do train-test
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, test_size=0.2)
model.fit(train_X, train_y)
predict_y = model.predict(test_X)
print("Train-Test Accuracy: {:3.2f}".format(accuracy_score(test_y, predict_y)))

# do the 5-fold cross validation
scores = cross_val_score(model, X, y, cv=5)
print("Fold Accuracies: {}".format(scores))
print("XV Accuracy: {:3.2f}".format(scores.mean()))

Train-Test Accuracy: 1.00
Fold Accuracies: [0.97 0.97 0.93 0.97 1.00]
XV Accuracy: 0.97


# Model Comparison

We now have two different kinds of models, decision trees and k-NN, we can use to do classification.

Let’s work our way through an example using the dataset ‘wdbc’ and compare the model performance of each of the models on that data set:

* Build optimal tree and KNN models using grid search
* Compute the accuracy for the classifiers
* Print out the confusion matrix for each classifier
* Print out the confidence interval for each classifier
* Decide if the difference between classifiers is statistically significant or not.

## Set Up

In [3]:
# basic data routines
import pandas as pd

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# model evaluation routines
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from assets.confint import classification_confint

# get data
df = pd.read_csv(notes_home+"assets/wdbc.csv")
df = df.drop(['ID'],axis=1)

# format training data for sklean
X  = df.drop(['Diagnosis'],axis=1)
actual_y = df['Diagnosis']

## Decision Trees

In [4]:
# decision trees
model = DecisionTreeClassifier(random_state=1)

# grid search
param_grid = {'max_depth': list(range(1,21)), 'criterion': ['entropy','gini'] }
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(X, actual_y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(actual_y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

# build the confusion matrix
labels = ['M', 'B']
cm = confusion_matrix(actual_y, predict_y, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Grid Search: best parameters: {'criterion': 'entropy', 'max_depth': 4}
Accuracy: 0.98 (0.97,0.99)
Confusion Matrix:
     M    B
M  210    2
B    7  350


## KNN

In [5]:
# KNN
model = KNeighborsClassifier()

# grid search
param_grid = {'n_neighbors': list(range(1,51))}
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(X, actual_y)
print("Grid Search: best parameters: {}".format(grid.best_params_))

# accuracy of best model with confidence interval
best_model = grid.best_estimator_
predict_y = best_model.predict(X)
acc = accuracy_score(actual_y, predict_y)
lb,ub = classification_confint(acc,X.shape[0])
print("Accuracy: {:3.2f} ({:3.2f},{:3.2f})".format(acc,lb,ub))

# build the confusion matrix
labels = ['B', 'M']
cm = confusion_matrix(actual_y, predict_y, labels=labels)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:\n{}".format(cm_df))

Grid Search: best parameters: {'n_neighbors': 14}
Accuracy: 0.94 (0.92,0.96)
Confusion Matrix:
     B    M
B  349    8
M   26  186


## Performance Comparison and Model Selection

The confidence intervals for the decision tree and the K-NN classifier do not overlap.  That means here the decision tree is truly the better model.